In [1]:
#************************************************************************************
# Rezwan Matin
# EE5321 – Project
# Filename: SVM_TESS_1.ipynb
# Date: 01/28/20
#
# Objective:
# 13 MFCCs (mean) and 13 MFCCs (standard deviation), ZCR.
#
#*************************************************************************************

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa as rosa
import glob
import os
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.metrics import confusion_matrix

In [2]:
# Save directory path in 'path'
path = r'C:\Books\Texas State Books\Fall 2019\Thesis A\Corpus\Simulated\TESS\All'

# Declare a dummy Numpy array (row vector)
result_array = np.empty([1,28])

# Create a list of audio file names 'file_list'
file_list = os.listdir(path)

i=0

for filename in file_list:
    
    # Read WAV file. 'rosa.core.load' returns sampling frequency in 'fs' and audio signal in 'sig'
    sig, fs = rosa.core.load(path + '\\' + file_list[i], sr=None)
    
    # 'rosa.feature.mfcc' extracts n_mfccs from signal and stores it into 'mfcc_feat'
    mfcc_feat = rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=13)
    
    # Calculate the average mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    avg_mfcc_feat = np.mean(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=13).T,axis=0)
    
    # Calculate the standard deviation of mfcc (utterance-level features) using 'rosa.feat.mfcc()' and 'np.std' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    std_mfcc_feat = np.std(rosa.feature.mfcc(y=sig, sr=fs, n_mfcc=13).T,axis=0)
    
    # Calculate the average zero crossing rate (utterance-level feature) using 'rosa.feat.zero_crossing_rate()' and 'np.mean' method. '.T' transposes the rows and columns. 'axis=0' indicates average is calculated column-wise
    zcross_feat = rosa.feature.zero_crossing_rate(sig)
    avg_zcross_feat = np.mean(rosa.feature.zero_crossing_rate(y=sig).T,axis=0)
    
    # Append the three 1D arrays into a single 1D array called 'feat'.
    feat0 = np.append(avg_mfcc_feat, std_mfcc_feat, axis=0)
    
    feat1 = np.append(feat0, avg_zcross_feat, axis=0)
    
    # Save emotion label from file name. 'path' contains directory's address, 'file_list' contains file name, and '\\' joins the two to form file's address
    label = os.path.splitext(os.path.basename(path + '\\' + file_list[i]))[0].split('_')[1]
    
    # Create a new Numpy array 'sample' to store features along with label
    sample = np.insert(feat1, obj=27, values=label)
    
    result_array = np.append(result_array, sample)
    
    i+=1

# Print out the 1D Numpy array
result_array

array([8.65493354e-316, 9.23902758e-322, 0.00000000e+000, ...,
       1.83165790e+001, 1.92256627e-001, 8.00000000e+000])

In [3]:
result_array.shape

(78428,)

In [4]:
# Convert 1D Numpy array to 2D array
result_array = np.reshape(result_array, (i+1,-1))

# Delete first dummy row from 2D array
result_array = np.delete(result_array, 0, 0)

# Print final 2D Numpy array 
result_array.shape

(2800, 28)

In [5]:
# Transfer Numpy array data to a Pandas dataframe
df = pd.DataFrame(data=result_array)
# Label only the last (target) column
df = df.rename({27: "Emotion"}, axis='columns')
# Rename class labels
df['Emotion'].replace({1.0: "Neutral", 3.0: "Happy", 4.0: "Sad", 5.0: "Angry", 6.0: "Fearful", 7.0: "Disgust", 8.0: "Surprised"}, inplace=True)
# Reset row (sample) indexing
df = df.reset_index(drop=True)
df.head(12)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,Emotion
0,-528.499228,86.100738,17.797453,1.335589,14.801382,12.804018,-4.624410,-4.640589,-10.816754,2.238214,...,19.287005,18.853138,18.208855,17.699930,13.491363,13.459267,11.978442,11.083434,0.068957,Neutral
1,-497.019387,77.665822,25.753440,-4.328508,2.269791,7.581418,-0.116068,-3.125772,-15.357324,1.173361,...,21.565630,18.630865,17.702558,14.180838,16.026444,15.134670,12.047095,12.145738,0.068990,Neutral
2,-500.522992,84.506182,20.194522,2.381852,14.780523,13.248763,-15.450830,-10.223414,-14.478630,-2.732910,...,17.871935,23.471593,16.943770,12.932126,14.109494,13.459020,10.554518,11.408428,0.082913,Neutral
3,-509.476734,73.102157,17.194391,12.129123,10.979809,3.319019,-1.237639,-7.961328,-14.416996,-5.655711,...,18.250876,17.559024,18.952916,11.903677,12.268972,12.243747,9.831696,10.863734,0.095975,Neutral
4,-535.158399,64.858644,18.889478,18.277332,18.269866,9.174235,-3.113041,-0.781078,-7.781502,2.094210,...,19.733857,21.821082,17.682783,15.053526,15.076096,14.410716,10.646933,11.153046,0.092968,Neutral
5,-511.694298,86.179209,21.385005,-0.183428,10.904503,12.704529,-8.055137,-8.335903,-16.956759,-1.713734,...,19.255290,19.512015,18.847285,13.872007,14.338249,11.961451,8.485458,10.048277,0.080454,Neutral
6,-515.520162,86.419970,23.480510,5.978081,11.823975,14.172115,-10.100084,-8.877399,-15.015842,-3.209784,...,21.815618,24.862090,19.186184,13.204553,15.943345,10.889276,7.445129,9.440382,0.082897,Neutral
7,-526.924232,86.073802,28.716866,2.332954,9.453578,13.863026,-5.956343,-9.159200,-15.009140,-1.574089,...,19.121752,22.482487,21.383344,13.628324,15.574618,13.938064,8.724893,11.093910,0.082504,Neutral
8,-536.702535,95.352169,26.064563,-4.630669,4.179155,15.128372,-5.022700,-9.922548,-15.720885,0.523983,...,21.963131,22.153071,20.688545,14.354853,14.919949,12.116925,9.521653,10.865204,0.070104,Neutral
9,-504.185201,69.966657,24.419953,11.739514,10.443279,7.342729,-5.722229,-2.698441,-15.555375,-4.689976,...,21.351309,22.532147,15.464300,16.490917,14.397415,13.324529,7.254590,9.920552,0.094280,Neutral


In [6]:
df['Emotion'].value_counts()

Disgust      400
Happy        400
Angry        400
Neutral      400
Sad          400
Surprised    400
Fearful      400
Name: Emotion, dtype: int64

In [8]:
# Extract target feature 'Emotion' in a vector y. Indexing from 0
y = df.iloc[0:2800, 27].values
# Extract features 'buying' and 'safety' in a vector X. Indexing from 0
X = df.iloc[0:2800, list(range(27))].values
print(y)

['Neutral' 'Neutral' 'Neutral' ... 'Surprised' 'Surprised' 'Surprised']


In [9]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1, stratify=y)

# Create an object 'sc'
sc = StandardScaler()

# Scale training data
sc.fit(X_train)
X_train_std = sc.transform(X_train)

In [10]:
# Create SVM model with hyperparameters
#svm = SVC(kernel='sigmoid', C=10.0, random_state=1)

# Best settings from GridSearchCV
svm = SVC(kernel='rbf', C=10.0, gamma=0.1, random_state=1)

# This is training the model
svm.fit(X_train_std, y_train)

SVC(C=10.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [11]:
# Scale test data
sc.fit(X_test)
X_test_std = sc.transform(X_test)

# Test the model data
y_pred = svm.predict(X_test_std)

In [12]:
# Print out the performance metrics
print('Misclassified samples: %d' % (y_test != y_pred).sum())
print('Training Accuracy: %.2f' % svm.score(X_train_std, y_train))
#print('Test Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Test Accuracy: %.2f' % svm.score(X_test_std, y_test))

# Print out the performance metrics in a text file.
#text_file = open("Emotion Classifier Performance Metrics.txt", "w")
#text_file.write('Misclassified samples: %d' % (y_test != y_pred).sum())
#text_file.write('Training Accuracy: %.2f' % svm.score(X_train_std, y_train))
#text_file.write('Test Accuracy: %.2f' % svm.score(X_test_std, y_test))
#text_file.close()

Misclassified samples: 10
Training Accuracy: 1.00
Test Accuracy: 0.99


In [13]:
fs

24414